First things first let's initialise our environment.

*Run the cell below to import our libraries and initialise our LUSID client*

In [32]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import uuid

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')

LUSID Environment Initialised


![Initailise LUSID](img/multiplecurrencies-initialise.gif)

## 1) Set up our Scope

In [33]:
scope_id = str(uuid.uuid4())[:4]
scope_name = 'UK_High_Growth_Equities_Fund'
scope = '{}_{}'.format(scope_name, scope_id)
print (scope)

UK_High_Growth_Equities_Fund_fa7e


![Scope](img/multiplecurrencies-scope.gif)

## 2) Load Instruments

In [34]:
instrument_universe = pd.read_csv('data/multiplecurrencies-instruments.csv')
instrument_universe.head(n=10)

instrument_name client_internal currency          isin          figi  \
0     Amazon_Nasdaq_AMZN    imd_34634534      USD  US0231351067  BBG000BVPXP1   
1      Apple_Nasdaq_AAPL    imd_35345345      USD  US0378331005  BBG000B9XVV8   
2   USTreasury_2.00_2021    imd_34535347      USD  US912828U816  BBG00FN3B5K8   
3  USTreasury_6.875_2025    imd_34534539      USD  US912810EV62  BBG000DQQNJ8   

  exchange_code          country_issue            ticker market_sector  \
0            UN  united_states_america              AMZN        equity   
1            UN  united_states_america              AAPL        equity   
2        BERLIN  united_states_america      T 2 12/31/21          govt   
3      NEW YORK  united_states_america  T 6.875 08/15/25          govt   

   security_type  coupon  
0   common_stock     NaN  
1   common_stock     NaN  
2  us_government   2.000  
3  us_government   6.875

In [35]:
# Initialise our dictionary to hold our instrument definitions
instrument_definitions = {}

# Iterate over our instrument universe
for index, instrument in instrument_universe.iterrows():

    # Create our ISIN property using the default ISIN property definition
    isin_property = models.InstrumentProperty(
        key='Instrument/default/Isin',
        value=models.PropertyValue(
            label_value=instrument['isin']))
    
    # Create our ticker property using the default ticker property definition
    ticker_property = models.InstrumentProperty(
        key='Instrument/default/Ticker',
        value=models.PropertyValue(
            label_value=instrument['ticker']))

    # Create the definition for our instrument
    instrument_definitions[instrument['instrument_name']] = models.InstrumentDefinition(
        name=instrument['instrument_name'],
        identifiers={'Figi': instrument['figi']},
        properties=[
            isin_property,
            ticker_property
        ]
    )

# Call LUSID to upsert our instrument defintions
response = client.upsert_instruments(requests=instrument_definitions)

# Pretty print the response from LUSID
prettyprint.instrument_response(response, identifier='Figi')

Instrument Successfully Upserted: Amazon_Nasdaq_AMZN
Figi ID: BBG000BVPXP1
LUSID Instrument ID: LUID_I6M8YNQN


Instrument Successfully Upserted: USTreasury_2.00_2021
Figi ID: BBG00FN3B5K8
LUSID Instrument ID: LUID_0BU9BXC2


Instrument Successfully Upserted: Apple_Nasdaq_AAPL
Figi ID: BBG000B9XVV8
LUSID Instrument ID: LUID_K8YYP5FU


Instrument Successfully Upserted: USTreasury_6.875_2025
Figi ID: BBG000DQQNJ8
LUSID Instrument ID: LUID_S5F04ES0


4  instruments upserted successfully
0  instrument upsert failures


![Instruments](img/multiplecurrencies-instruments.gif)

## 3) Create our Base Fund Portfolio

In [36]:
base_portfolio_name = '{}_base_fund'.format(scope_name)
print (base_portfolio_name)

UK_High_Growth_Equities_Fund_base_fund


In [37]:
# The date our portfolios were first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=365))

# Create the request to add our portfolio
transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name=base_portfolio_name,
    code=base_portfolio_name,
    base_currency='GBP',
    description='The portfolio to hold our base fund',
    created=portfolio_creation_date)

# Call LUSID to create our portfolio
portfolio_response = client.create_portfolio(
    scope=scope,
    create_request=transaction_portfolio_request)

# Pretty print the response from LUSID
prettyprint.portfolio_response(portfolio_response)

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_fa7e
Code: UK_High_Growth_Equities_Fund_base_fund
Portfolio Effective From: 2018-02-20 11:43:46.585954+00:00
Portfolio Created On: 2019-02-20 11:43:46.788180+00:00



![Base-Fund](img/multiplecurrencies-basefundportfolio.gif)

## 4) Set our Holdings

In [38]:
#Import our holdings
holdings = pd.read_csv('data/multiplecurrencies-holdings.csv')
holdings.head()

instrument_name client_internal          isin          figi  \
0     Amazon_Nasdaq_AMZN    imd_34634534  US0231351067  BBG000BVPXP1   
1      Apple_Nasdaq_AAPL    imd_35345345  US0378331005  BBG000B9XVV8   
2   USTreasury_2.00_2021    imd_34535347  US912828U816  BBG00FN3B5K8   
3  USTreasury_6.875_2025    imd_34534539  US912810EV62  BBG000DQQNJ8   

   quantity  unit_cost   total_cost currency  
0      5000    1550.00   7750000.00      USD  
1     49567     190.00   9417730.00      USD  
2    121543      99.25  12063142.75      USD  
3     98444     140.98  13878635.12      USD

In [39]:
holding_adjustments = []

holdings_effective_date = datetime.now(pytz.UTC) - timedelta(days=7)

for row, holding in holdings.iterrows():

    holding_adjustments.append(
        models.AdjustHoldingRequest(
            instrument_identifiers={
                    'Instrument/default/Figi': holding['figi']},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=holding['quantity'],
                        cost=models.CurrencyAndAmount(
                            amount=holding['total_cost'],
                            currency=holding['currency']),
                        portfolio_cost=holding['total_cost'],
                        price=holding['unit_cost'])
                ]
        )
    )
    
# Call LUSID to set our holdings
response = client.set_holdings(
    scope=scope,
    code=base_portfolio_name,
    effective_at=holdings_effective_date,
    holding_adjustments=holding_adjustments)

prettyprint.set_holdings_response(response, scope, base_portfolio_name)

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_fa7e
Code: UK_High_Growth_Equities_Fund_base_fund
Holdings Effective From: 2019-02-13 11:43:51.531548+00:00
Holdings Created On: 2019-02-20 11:43:51.809918+00:00



![Base-Fund-Hodlings](img/multiplecurrencies-basefundportfolioholdings.gif)

## 5) Add Corporate Actions

In [40]:
corporate_action_source_id = '{}_corporate_action_source'.format(base_portfolio_name)

corporate_action_request = models.CreateCorporateActionSourceRequest(
    scope=scope,
    code=corporate_action_source_id)

response = client.create_corporate_action_source(
    request=corporate_action_request)
    
print (response)

{'additional_properties': {}, 'href': None, 'id': <lusid.models.resource_id.ResourceId object at 0x7f91cd99ec18>, 'version': None, 'links': [<lusid.models.link.Link object at 0x7f91cd99eef0>]}


![Corporate-Action-Source](img/multiplecurrencies-corporateactionsource.gif)

In [41]:
corporate_actions = pd.read_csv('data/multiplecurrencies-corporateactions.csv')
corporate_actions.head()

code                 announcement_date  \
0  5943592342  2019-01-29 12:04:20.970603+00:00   

                     execution_date                       record_date  \
0  2019-02-08 12:04:20.970603+00:00  2019-02-11 12:04:20.970603+00:00   

                       payment_date input_instrument_figi  \
0  2019-02-14 12:04:20.970603+00:00          BBG000B9XVV8   

  input_instrument_internal  input_units_factor  input_cost_factor  \
0              imd_34535347                   1                  1   

   output_instrument_figi output_instrument_internal  output_units_factor  \
0                     NaN                    CCY_GBP                    2   

   output_cost_factor  dividend_yield action_description  
0                   2          0.0017           dividend

In [42]:
actions = []

for row, action in corporate_actions.iterrows():
    
    luid = client.get_instrument(
        identifier_type='Figi',
        identifier=action['input_instrument_figi']).lusid_instrument_id
    
    transition = models.CorporateActionTransition(
        input_transition=models.CorporateActionTransitionComponent(
            instrument_uid=luid,
            units_factor=1,
            cost_factor=0),
        output_transitions=[
            models.CorporateActionTransitionComponent(
            instrument_uid=action['output_instrument_internal'],
            units_factor=action['dividend_yield'],
            cost_factor=0),
        ])
    
    actions.append(
        models.CreateCorporateAction(
            corporate_action_code=action['code'],
            announcement_date=datetime.now(pytz.UTC)+timedelta(days=1),
            ex_date=datetime.now(pytz.UTC)+timedelta(days=1),
            record_date=datetime.now(pytz.UTC)+timedelta(days=1),
            payment_date=datetime.now(pytz.UTC)+timedelta(days=1),
            transitions=[transition])
    )

response = client.batch_upsert_corporate_actions(
    scope=scope,
    code=corporate_action_source_id,
    actions=actions)

print (response)

{'additional_properties': {}, 'href': None, 'values': {'5943592342': <lusid.models.corporate_action.CorporateAction object at 0x7f91ce337d30>}, 'failed': {}, 'links': [<lusid.models.link.Link object at 0x7f91cd99e3c8>]}


In [43]:
response = client.upsert_portfolio_details(
    scope=scope,
    code=base_portfolio_name,
    details=models.CreatePortfolioDetails(
        corporate_action_source_id=models.ResourceId(
            scope=scope,
            code=corporate_action_source_id),
        base_currency='GBP'))

print (response)

{'additional_properties': {}, 'href': 'https://api.lusid.com/api/transactionportfolios/UK_High_Growth_Equities_Fund_fa7e/UK_High_Growth_Equities_Fund_base_fund/details?effectiveAt=2019-02-20T11%3A44%3A00.6365620%2B00%3A00&asAt=2019-02-20T11%3A44%3A03.3683300%2B00%3A00', 'origin_portfolio_id': <lusid.models.resource_id.ResourceId object at 0x7f91cd9c05c0>, 'version': <lusid.models.version.Version object at 0x7f91cd9d0780>, 'base_currency': 'GBP', 'links': [<lusid.models.link.Link object at 0x7f91cda0d588>]}


![Corporate-Actions](img/multiplecurrencies-corporateactions.gif)

## 6) Value our Base Fund Portfolio

### a) Analytic store

In [44]:
prices = pd.read_csv('data/multiplecurrencies-prices.csv')
prices.head()

time    price   type currency        instrument_name instrument_figi  \
0  18/02/2019  1622.65  close      USD     Amazon_Nasdaq_AMZN    BBG000BVPXP1   
1  18/02/2019   170.80  close      USD      Apple_Nasdaq_AAPL    BBG000B9XVV8   
2  18/02/2019    98.00  close      USD   USTreasury_2.00_2021    BBG00FN3B5K8   
3  18/02/2019   142.36  close      USD  USTreasury_6.875_2025    BBG000DQQNJ8   
4  17/02/2019  1619.23  close      USD     Amazon_Nasdaq_AMZN    BBG000BVPXP1   

  instrument_internal  
0        imd_34634534  
1        imd_34535347  
2        imd_34535347  
3        imd_34534539  
4        imd_34634534

In [45]:
times = prices['time'].unique()

for time in times:
    quote_date = datetime.strptime(time, '%d/%m/%Y')

    # Create analytics store request
    analytics_store_request = models.CreateAnalyticStoreRequest(
        scope=scope,
        date_property=quote_date)

    # Call LUSID to create our analytics store
    client.create_analytic_store(request=analytics_store_request)
    
    instrument_analytics = []
    for row, quote in prices.loc[prices['time'] == time].iterrows():
        
        luid = client.get_instrument(
            identifier_type='Figi',
            identifier=quote['instrument_figi']).lusid_instrument_id
        
        instrument_analytics.append(
            models.InstrumentAnalytic(
                instrument_uid=luid,
                value=quote['price']))
        
    response = client.set_analytics(
        scope=scope,
        year=quote_date.year,
        month=quote_date.month,
        day=quote_date.day,
        data=instrument_analytics)
    
    print (response.key)
    

Datetime with no tzinfo will be considered UTC.
Datetime with no tzinfo will be considered UTC.


{'additional_properties': {}, 'scope': 'UK_High_Growth_Equities_Fund_fa7e', 'date_property': datetime.datetime(2019, 2, 18, 0, 0, tzinfo=<FixedOffset '+00:00'>)}
{'additional_properties': {}, 'scope': 'UK_High_Growth_Equities_Fund_fa7e', 'date_property': datetime.datetime(2019, 2, 17, 0, 0, tzinfo=<FixedOffset '+00:00'>)}


![Analytic-Stores](img/multiplecurrencies-analyticstores.gif)

### b) Aggregation

In [46]:
aggregation_request = models.AggregationRequest(
    recipe_id=models.ResourceId(
        scope=scope,
        code='default'),
        effective_at=datetime.now(pytz.UTC),
        metrics=[
            models.AggregateSpec(key='Holding/default/SubHoldingKey',
            op='Value'),
            models.AggregateSpec(key='Instrument/default/Name',
            op='Value'),
            models.AggregateSpec(key='Holding/default/Units',
            op='sum'),
            models.AggregateSpec(key='Holding/default/Cost',
            op='sum'),
            models.AggregateSpec(key='Holding/default/PV',
            op='sum'),
        ],
        group_by=[
            'Holding/default/SubHoldingKey'
        ])

response = client.get_aggregation_by_portfolio(
    scope=scope,
    code=base_portfolio_name,
    request=aggregation_request)

print(response.data)

[{'Holding/default/SubHoldingKey': 'LusidInstrumentId=LUID_I6M8YNQN/USD', 'Instrument/default/Name': 'Amazon_Nasdaq_AMZN', 'Sum(Holding/default/Units)': 5000.0, 'Sum(Holding/default/Cost)': 7750000.0, 'Sum(Holding/default/PV)': 8113250.0}, {'Holding/default/SubHoldingKey': 'LusidInstrumentId=LUID_K8YYP5FU/USD', 'Instrument/default/Name': 'Apple_Nasdaq_AAPL', 'Sum(Holding/default/Units)': 49567.0, 'Sum(Holding/default/Cost)': 9417730.0, 'Sum(Holding/default/PV)': 8466043.6}, {'Holding/default/SubHoldingKey': 'LusidInstrumentId=LUID_0BU9BXC2/USD', 'Instrument/default/Name': 'USTreasury_2.00_2021', 'Sum(Holding/default/Units)': 121543.0, 'Sum(Holding/default/Cost)': 12063142.75, 'Sum(Holding/default/PV)': 11911214.0}, {'Holding/default/SubHoldingKey': 'LusidInstrumentId=LUID_S5F04ES0/USD', 'Instrument/default/Name': 'USTreasury_6.875_2025', 'Sum(Holding/default/Units)': 98444.0, 'Sum(Holding/default/Cost)': 13878635.12, 'Sum(Holding/default/PV)': 14014487.84}]


![Analytic-Stores](img/multiplecurrencies-aggregatedbase.gif)

## 7) Securitise our Base Fund Portfolio

### a) Securitiese with x units

In [47]:
# Create our request to define a new property
property_request = models.CreatePropertyDefinitionRequest(
    domain='Instrument',
    scope=scope,
    code='total_circulation',
    value_required=False,
    display_name='total_circulation',
    data_type_id=models.ResourceId(scope='default', code='number'))

# Call LUSID to create our new property
response = client.create_property_definition(definition=property_request)

# Grab the key off the response to use when referencing this property in other LUSID calls
circulation_property_key = response.key

# Pretty print our key
prettyprint.heading('Circulation Property Key', circulation_property_key)

Circulation Property Key: Instrument/UK_High_Growth_Equities_Fund_fa7e/total_circulation


![Analytic-Stores](img/multiplecurrencies-circulationproperty.gif)

In [48]:
instrument_definition = models.InstrumentDefinition(
    name=base_portfolio_name,
    identifiers={'ClientInternal': base_portfolio_name},
    properties=[models.InstrumentProperty(
        key=circulation_property_key,
        value=models.PropertyValue(
            metric_value=models.MetricValue(
                value=40000.00
                )))],
    look_through_portfolio_id=models.ResourceId(
        scope=scope,
        code=base_portfolio_name))

response = client.upsert_instruments(requests={'look-through':instrument_definition}
    )

In [49]:
print (response.values)

{'look-through': <lusid.models.instrument.Instrument object at 0x7f91cda07828>}


![Analytic-Stores](img/multiplecurrencies-securitisebase.gif)

### b) Calculalate price from aggregation and store in the aggregation store

In [50]:
aggregation_request = models.AggregationRequest(
    recipe_id=models.ResourceId(
        scope=scope,
        code='default'),
        effective_at=datetime.now(pytz.UTC),
        metrics=[
            models.AggregateSpec(key='Holding/default/Cost',
            op='sum'),
            models.AggregateSpec(key='Holding/default/PV',
            op='sum'),
        ],
        group_by=[
            'Portfolio/default/Name'
        ])

response = client.get_aggregation_by_portfolio(
    scope=scope,
    code=base_portfolio_name,
    request=aggregation_request)

print (response.data)

total_value = response.data[0]['Sum(Holding/default/PV)']
total_cost = response.data[0]['Sum(Holding/default/Cost)']

[{'Sum(Holding/default/Cost)': 43109507.87, 'Sum(Holding/default/PV)': 42504995.44}]


In [52]:
instrument_analytics = []

quote_date = datetime.now(pytz.UTC) - timedelta(days=3)

base_fund_instrument = client.get_instrument(
    identifier_type='ClientInternal',
    identifier=base_portfolio_name,
    instrument_property_keys=[circulation_property_key])

luid = base_fund_instrument.lusid_instrument_id
total_circulation = base_fund_instrument.properties[0].value
price = total_value / total_circulation

print (price)

instrument_analytics.append(
    models.InstrumentAnalytic(
        instrument_uid=luid,
        value=price))

response = client.set_analytics(
        scope=scope,
        year=quote_date.year,
        month=quote_date.month,
        day=quote_date.day,
        data=instrument_analytics)

print (response.key)

1062.6248859999998
{'additional_properties': {}, 'scope': 'UK_High_Growth_Equities_Fund_fa7e', 'date_property': datetime.datetime(2019, 2, 17, 0, 0, tzinfo=<FixedOffset '+00:00'>)}


![Base-Fund-Price](img/multiplecurrencies-basefundprice.gif)

## 8) Create Foreign Currency Portfolios

In [53]:
fund_currencies = ['AUD', 'GBP', 'USD', 'EUR', 'JPY']

for currency in fund_currencies:
    
    portfolio_name = '{}_{}'.format(base_portfolio_name, currency)
    
    # Create the request to add our portfolio
    transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
        display_name=portfolio_name,
        code=portfolio_name,
        base_currency=currency,
        description='The portfolio to hold our base fund in currency {}'.format(currency),
        created=portfolio_creation_date)

    # Call LUSID to create our portfolio
    portfolio_response = client.create_portfolio(
        scope=scope,
        create_request=transaction_portfolio_request)

    # Pretty print the response from LUSID
    prettyprint.portfolio_response(portfolio_response)

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_fa7e
Code: UK_High_Growth_Equities_Fund_base_fund_AUD
Portfolio Effective From: 2018-02-20 11:43:46.585954+00:00
Portfolio Created On: 2019-02-20 11:45:03.512219+00:00

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_fa7e
Code: UK_High_Growth_Equities_Fund_base_fund_GBP
Portfolio Effective From: 2018-02-20 11:43:46.585954+00:00
Portfolio Created On: 2019-02-20 11:45:03.806548+00:00

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_fa7e
Code: UK_High_Growth_Equities_Fund_base_fund_USD
Portfolio Effective From: 2018-02-20 11:43:46.585954+00:00
Portfolio Created On: 2019-02-20 11:45:04.109113+00:00

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_fa7e
Code: UK_High_Growth_Equities_Fund_base_fund_EUR
Portfolio Effective From: 2018-02-20 11:43:46.585954+00:00
Portfolio Created On: 2019-02-20 11:45:04.379126+00:00

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_fa7e
Code: UK_High_Growth_Equities_Fund_base_fund_JPY


![Foreign-Currency-Portfolios](img/multiplecurrencies-foreigncurrencyportfolios.gif)

## 9) Add Base Fund Units

In [57]:
holding_adjustments = []

holdings_effective_date = datetime.now(pytz.UTC) - timedelta(days=7)

holding_adjustments.append(
    models.AdjustHoldingRequest(
        instrument_identifiers={
                'Instrument/default/ClientInternal': base_portfolio_name},
            tax_lots=[
                models.TargetTaxLotRequest(
                    units=total_circulation/len(fund_currencies),
                    cost=models.CurrencyAndAmount(
                        amount=total_cost/len(fund_currencies),
                        currency='USD'),
                    portfolio_cost=total_cost/len(fund_currencies),
                    price=total_cost/total_circulation)
            ]
    )
)

for currency in fund_currencies:
    # Call LUSID to set our holdings
    response = client.set_holdings(
        scope=scope,
        code='{}_{}'.format(base_portfolio_name, currency),
        effective_at=holdings_effective_date,
        holding_adjustments=holding_adjustments)

    prettyprint.set_holdings_response(response, scope, '{}_{}'.format(base_portfolio_name, currency))

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_fa7e
Code: UK_High_Growth_Equities_Fund_base_fund_AUD
Holdings Effective From: 2019-02-17 11:45:06.446182+00:00
Holdings Created On: 2019-02-20 11:46:05.853153+00:00

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_fa7e
Code: UK_High_Growth_Equities_Fund_base_fund_GBP
Holdings Effective From: 2019-02-17 11:45:06.446182+00:00
Holdings Created On: 2019-02-20 11:46:06.215880+00:00

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_fa7e
Code: UK_High_Growth_Equities_Fund_base_fund_USD
Holdings Effective From: 2019-02-17 11:45:06.446182+00:00
Holdings Created On: 2019-02-20 11:46:06.597755+00:00

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_fa7e
Code: UK_High_Growth_Equities_Fund_base_fund_EUR
Holdings Effective From: 2019-02-17 11:45:06.446182+00:00
Holdings Created On: 2019-02-20 11:46:06.968405+00:00

Holdings Successfully Set fo

![Foreign-Currency-Portfolios](img/multiplecurrencies-foreigncurrencyportfoliosholdbase.gif)

## 10) Add Hedging Transactions

In [58]:
response = client.get_holdings(
    scope=scope,
    code='{}_{}'.format(base_portfolio_name, 'GBP'))
    
print (response.values[0])

{'additional_properties': {}, 'instrument_uid': 'LUID_1E4YQGP1', 'sub_holding_keys': [], 'properties': [], 'holding_type': 'P', 'units': 8000.0, 'settled_units': 8000.0, 'cost': <lusid.models.currency_and_amount.CurrencyAndAmount object at 0x7f91cd9c0470>, 'cost_portfolio_ccy': <lusid.models.currency_and_amount.CurrencyAndAmount object at 0x7f91cd9c01d0>, 'transaction': None}


In [59]:
aggregation_request = models.AggregationRequest(
    recipe_id=models.ResourceId(
        scope=scope,
        code='default'),
        effective_at=quote_date,
        metrics=[
            models.AggregateSpec(key='Holding/default/SubHoldingKey',
            op='Value'),
            models.AggregateSpec(key='Instrument/default/Name',
            op='Value'),
            models.AggregateSpec(key='Holding/default/Units',
            op='sum'),
            models.AggregateSpec(key='Holding/default/Cost',
            op='sum'),
            models.AggregateSpec(key='Holding/default/PV',
            op='sum'),
        ],
        group_by=[
            'Holding/default/SubHoldingKey'
        ])

response = client.get_aggregation_by_portfolio(
    scope=scope,
    code='{}_{}'.format(base_portfolio_name, 'JPY'),
    request=aggregation_request)

response = client.get_aggregation_by_portfolio(
    scope=scope,
    code=base_portfolio_name,
    request=aggregation_request)

print(response.data)

ErrorResponseException: A price was not found for instrument

## 11) Value Currency Portfolios

## 12) Securitise Currency Portfolios

## 13) Create Share Class Portfolios

## 15) Add Currency Fund Holdings

## 16) Value Each Share Class

## 17) Add transactions to end portfolio

## 18) Value each portfolio (bubble up)